# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг <start>  
    - еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы  
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

In [1]:
from string import punctuation
from collections import Counter
import numpy as np
from razdel import tokenize as razdel_tokenize
from nltk.tokenize import sent_tokenize
from scipy.sparse import lil_matrix
from sklearn.model_selection import train_test_split

Возьмем для обучения языковой модели корпус текстов Л.Н.Толстого ("Война и мир" и "Анна Каренина").

In [2]:
corpus = open('tolstoy.txt', encoding='utf8').read()

In [3]:
print('Объем корпуса в символах:', len(corpus))

Объем корпуса в символах: 4600956


In [4]:
sents_tokenized = sent_tokenize(corpus)
print('Объем корпуса в предложениях:', len(sents_tokenized))

Объем корпуса в предложениях: 50531


Далее нормализуем предложения (удаляем пунктуацию, приводим к нижнему регистру) и добавляем к токенизированным предложениям два тега в начало \<start> \<start> (чтобы строить вероятность по двум предыдущим токенам) и один тег \<end> в конец. Далее выделяем часть корпуса на train, чтобы посчитать перплексию (выборку для train возьмем небольшую, около 0.05%).

In [5]:
def normalize(text):
    text_normalized = [word.text.strip(punctuation) for word in razdel_tokenize(text)]
    text_normalized = [word.lower() for word in text_normalized if word]
    return text_normalized

In [6]:
sents_normalized = []
for sent in sents_tokenized:
    sent = normalize(sent)
    sent.insert(0, '<start>')
    sent.insert(0, '<start>')
    sent.append('<end>')
    sents_normalized.append(sent)

In [7]:
X, y = train_test_split(sents_normalized, test_size = 0.0005, shuffle=True)
print('Объем тестовой выборки:', len(y))

Объем тестовой выборки: 26


Составляем частнотности для 1-грамм, 2-грамм и 3-грамм для подсчета статистик.

In [8]:
def ngrammer(tokens, n):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [9]:
unigrams = Counter()
bigrams = Counter()
trigrams = Counter()

for sent in X:
    unigrams.update(sent)
    bigrams.update(ngrammer(sent, 2))
    trigrams.update(ngrammer(sent, 3))

Посмотрим на наиболее часто встречающиеся биграммы и триграммы.

In [10]:
for i in bigrams.most_common(10):
    print(i)

('<start> <start>', 50505)
('<start> —', 13968)
('<start> он', 2357)
('— сказал', 2324)
('<start> и', 1647)
('что он', 1484)
('<start> но', 1339)
('<start> она', 1291)
('и не', 1231)
('— сказала', 1118)


In [11]:
for i in trigrams.most_common(10):
    print(i)

('<start> <start> —', 13968)
('<start> <start> он', 2357)
('<start> <start> и', 1647)
('<start> <start> но', 1339)
('<start> <start> она', 1291)
('<start> <start> в', 1084)
('<start> <start> я', 1048)
('<start> — я', 883)
('— сказал он', 847)
('<start> — да', 761)


Составляем матрицу вероятностей.

In [12]:
matrix_corpus = lil_matrix((len(bigrams), len(unigrams)))

id2unigram = list(unigrams)
unigram2id = {word: i for i, word in enumerate(id2unigram)}

id2bigram = list(bigrams)
bigram2id = {word: i for i, word in enumerate(id2bigram)}

for ngram in trigrams:
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' ' + word2
    matrix_corpus[bigram2id[bigram], unigram2id[word3]] = (trigrams[ngram]/bigrams[bigram])

И теперь генерируем тексты с помощью модели: на вход изначально подаются токены \<start> \<start>, далее рандомно, но с учетом вероятностей, выбирается униграмма. Далее на вход подается токен \<start> и выбранный на предыдущем шаге токен, и снова выбирается следующий вероятный токен и т.д.

In [13]:
def generate(matrix, id_to_unigram, bigram_to_id, n=100, start='<start> <start>'):
    
    text = [[]]
    current_idx = bigram_to_id[start]
    count = 0
    
    for i in range(n):
        
        chosen = np.random.choice(list(range(matrix.shape[1])), p=matrix[current_idx].toarray()[0])
        text[count].append(id_to_unigram[chosen])
        
        if len(text[count]) == 1:
            current_idx = bigram_to_id['<start>' + ' ' + text[count][0]]
        
        if len(text[count]) > 1:
            current_idx = bigram_to_id[text[count][len(text[count])-2] + ' ' + text[count][len(text[count])-1]]
            
        if id_to_unigram[chosen] == '<end>':
            current_idx = bigram_to_id[start]
            count += 1
            text.append([]) 
            
    text = ' '.join([' '.join(sent) for sent in text]).replace('<end>', '\n')
    
    return text

In [29]:
for n in range(5):
    print(generate(matrix_corpus, id2unigram, bigram2id))
    print('\n')

она взглянула на него 
 — ну так так 
 — что я рискую злоупотребить вашим вниманием 
 но дамы невольно смеялись и князь андрей опять к ним французы то староста сказал что он делал и не могу наказать его ежели он сделает дурно и не успел еще бельяр скрыться из вида и вдруг отчаянно бросил ее и перешел на злоупотребления властей в соединенных штатах но анна не отвечая ничего ни слабости ничего я чувствую его 
 — у николеньки есть эта слабость что если бы наполеон успел уйти 
 — а ты 
 она видела 
 передовые ее низкие


пустить их и смягчить их возражения 
 в передней увидав пьера он сердито пробормотал что-то и левин был влюблен но — сказал болконский 
 — нет алексей александрович расспросил в чем состоит это благо 
 все выскажу ему все равно не запомню только что вышли 
 одно успокоительное рассуждение о том что облонский улыбался говоря это в такую минуту повторил князь андрей с презрением смотрел на смеющегося сперанского 
 человек этот сводится со всеми вместе 
 — думал князь андрей 

Теперь рассчитаем перплексию модели на тестовой выборке.

In [18]:
def perplexity(probas):
    p = np.exp(np.sum(probas))
    N = len(probas)
    return p**(-1/N) 

In [19]:
metrics = []

for sent in y:
    
    probs = []
    for trigram in ngrammer(sent, 3):
        word1, word2, word3 = trigram.split()
        bigram = word1 + ' ' + word2
        
        if bigram in bigrams and trigram in trigrams:
            probs.append(np.log(trigrams[trigram]/bigrams[bigram]))
        else:
            probs.append(np.log(0.00001))
    
    metrics.append(probs)

In [20]:
print(np.mean([perplexity(x) for x in metrics]))

12907.601228660009


## Задание № 2* (2 балла). 

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

Существует еще как минимум два способа решения проблемы несловарных слов.<br><br>
Первый из них применятся для систем с открытым словарем (то есть таких, в которых словарь задается вне обучающей выборки, поэтому в тренировочном сете появляются несловарные слова). Каждому слову в обучающей выборке, которое не встретилось в словаре, присваивается специальный токен \<OOV> (out of vocabulary) или \<UNK> (unknown), а дальше вероятности для этого токена рассчитываются также, как и для остальных слов в датасете.<br><br>
Второй способ применяется в ситуациях, когда словарь изначально не задан и собирается на основе обучающего сета. Чтобы словарь был не слишком объемный, можно избавиться от очень редких слов (можно задать, например, значение частотности, по которому будут отбираться слова, или заранее задать размерность словаря). Для таких слов так же, как и при первом способе, задается специальный токен \<OOV> или \<UNK> и для него считается вероятность.

2. Что такое сглаживание (smoothing)?

Сглаживание - это способ решения проблемы редких слов, не встретившихся в обучающем корпусе. Те слова (н-граммы) из тестовой выборки, которых не было в тренировочном датасете, в процессе обучения получают нулевую вероятность, хотя, конечно, не должны, так как все же в текстах встречаются. Чтобы предотвратить присваивание таким словам нулевой вероятности, существует несколько методов сглаживания. Самый простой способ - сглаживание Лапласа (это добавление 1 ко всем частотам в корпусе) или чуть расширенный метод - сглаживание add-k (это добавление не 1, а k, заданного от 0 до 1). Существуют также методы, суть которых сводится к тому, что вероятность не встретившихся н-грамм рассчитывается на основании (н-1)-грамм (Kneser-Ney smoothing).